Testing the minimap2 aligner
use conda environment "txomics"
download latest version using conda
You need to open vscode after switching to txomics environment and remap folder in the terminal

In [2]:
! minimap2 --version

2.22-r1101


Download some test FASTQ files
From Roach et. al., 2020
ERR3245465, L1 rep 2
ERR3245468, L3 rep 1
Download from ENA FTP server

In [3]:
! wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/005/ERR3245465/ERR3245465.fastq.gz
! wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/008/ERR3245468/ERR3245468.fastq.gz

--2024-10-17 11:05:44--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/005/ERR3245465/ERR3245465.fastq.gz
           => ‘ERR3245465.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.165|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/fastq/ERR324/005/ERR3245465 ... done.
==> SIZE ERR3245465.fastq.gz ... 479203040
==> PASV ... done.    ==> RETR ERR3245465.fastq.gz ... done.
Length: 479203040 (457M) (unauthoritative)

ERR3245465.fastq.gz 100%[===================>] 457.00M  38.3MB/s    in 13s     

2024-10-17 11:05:57 (36.0 MB/s) - ‘ERR3245465.fastq.gz’ saved [479203040]

--2024-10-17 11:05:58--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/008/ERR3245468/ERR3245468.fastq.gz
           => ‘ERR3245468.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.a

In [4]:
! mkdir data/
! mv ERR3245465.fastq.gz ERR3245468.fastq.gz data/

Create small fastq files with 1% of reads for testing

In [5]:
! lines=$(zcat data/ERR3245465.fastq.gz | wc -l) && zcat data/ERR3245465.fastq.gz | head -n $((((lines / 4) / 100) * 4)) \
    > 1pct.ERR3245465.fastq
! lines=$(zcat data/ERR3245468.fastq.gz | wc -l) && zcat data/ERR3245468.fastq.gz | head -n $((((lines / 4) / 100) * 4)) \
    > 1pct.ERR3245468.fastq


gzip: stdout: Broken pipe

gzip: stdout: Broken pipe


In [6]:
! mv 1pct* data/
# don't need !?

In [7]:
cp /home/carl/Code/datacore2024/genome_celegans/1pct* data/

General use command on minimap2 github
Can use gzip fasta and fastq files with minimap2
Some notes from reading the SAM format specification file:
Chimeric alignments are more common on long reads
Linear alignments do not involve strand switching

In [8]:
! minimap2 -a data/1pct_elegans.fa data/1pct.ERR3245465.fastq > test_align5.sam
! minimap2 -a data/1pct_elegans.fa data/1pct.ERR3245468.fastq > test_align8.sam

[M::mm_idx_gen::0.065*0.66] collected minimizers
[M::mm_idx_gen::0.094*0.78] sorted minimizers
[M::main::0.095*0.78] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.101*0.79] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.106*0.80] distinct minimizers: 166296 (91.68% are singletons); average occurrences: 1.131; average spacing: 5.330; total length: 1002861
[M::worker_pipeline::0.470*0.92] mapped 5903 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -a data/1pct_elegans.fa data/1pct.ERR3245465.fastq
[M::main] Real time: 0.474 sec; CPU: 0.436 sec; Peak RSS: 0.031 GB
[M::mm_idx_gen::0.075*0.44] collected minimizers
[M::mm_idx_gen::0.097*0.54] sorted minimizers
[M::main::0.097*0.54] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.100*0.55] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.103*0.56] distinct minimizers: 166296 (91.68% are si

Let's look at test_align.sam
I see a lot of bit 4 in the FLAG column, meaning that segment is unmapped.

In [9]:
with open('test_align8.sam', 'r') as fp:
    for line in fp.readlines():
        line = line.rstrip()
        line = line.split('\t')
        if line[1] == '2': 
            print(line)

There are no 2 flags in test_align.sam. 2 = each segment properly aligned according to the aligner. If I align more of the reads to more of the genome, will I get proper alignments?
Let's try 10%

In [8]:
! lines=$(zcat data/ERR3245465.fastq.gz | wc -l) && zcat data/ERR3245465.fastq.gz | head -n $((((lines / 4) / 10) * 4)) \
    > 10pct.ERR3245465.fastq
! lines=$(zcat data/ERR3245468.fastq.gz | wc -l) && zcat data/ERR3245468.fastq.gz | head -n $((((lines / 4) / 10) * 4)) \
    > 10pct.ERR3245468.fastq


gzip: stdout: Broken pipe

gzip: stdout: Broken pipe


In [14]:
! minimap2 -a data/1pct_elegans.fa 10pct.ERR3245465.fastq > test_align5.10.sam
! minimap2 -a data/1pct_elegans.fa 10pct.ERR3245468.fastq > test_align8.10.sam

[M::mm_idx_gen::0.052*0.76] collected minimizers
[M::mm_idx_gen::0.082*0.80] sorted minimizers
[M::main::0.082*0.80] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.086*0.81] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.090*0.81] distinct minimizers: 166296 (91.68% are singletons); average occurrences: 1.131; average spacing: 5.330; total length: 1002861
[M::worker_pipeline::2.278*1.34] mapped 59037 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -a data/1pct_elegans.fa 10pct.ERR3245465.fastq
[M::main] Real time: 2.283 sec; CPU: 3.048 sec; Peak RSS: 0.137 GB
[M::mm_idx_gen::0.058*0.97] collected minimizers
[M::mm_idx_gen::0.077*1.09] sorted minimizers
[M::main::0.077*1.09] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.081*1.08] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.084*1.08] distinct minimizers: 166296 (91.68% are singl

In [11]:
# lets try the entire fasta file and see if 2 shows up
! minimap2 -a data/1pct_elegans.fa data/ERR3245465.fastq.gz > test_align5.1pAll.sam
! minimap2 -a data/1pct_elegans.fa data/ERR3245468.fastq.gz > test_align8.1pAll.sam

[M::mm_idx_gen::0.018*0.80] collected minimizers
[M::mm_idx_gen::0.024*1.25] sorted minimizers
[M::main::0.024*1.25] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.026*1.23] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.027*1.22] distinct minimizers: 166296 (91.68% are singletons); average occurrences: 1.131; average spacing: 5.330; total length: 1002861
[M::worker_pipeline::8.496*1.72] mapped 590372 sequences
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -a data/1pct_elegans.fa data/ERR3245465.fastq.gz
[M::main] Real time: 8.523 sec; CPU: 14.606 sec; Peak RSS: 1.198 GB
[M::mm_idx_gen::0.021*1.04] collected minimizers
[M::mm_idx_gen::0.029*1.45] sorted minimizers
[M::main::0.029*1.45] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.031*1.42] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.033*1.40] distinct minimizers: 166296 (91.68% are s

Did not find any 2s with the full fasta file. Now try with the entire genome.

In [ ]:
! minimap2 -a data/1pct_elegans.fa data/ERR3245468.fastq.gz > test_align8.All.sam

Use look_sam.py. still didin't find any 2s. but why did 272 as a bit flag show up? that's not in the SAM documentation. See this thread:
https://github.com/lh3/minimap2/issues/893
But 272 does not show up in the SAM docs. So how do they know?

Let's use the same parameters used by Roach et. al., 2020 for minimap2
-ax splice -k14 -uf --secondary = no -G 25000 -t 24
Then got SAM files, converted to BAM with SAMtools view -b -F 2048